# 3.1 일반적인 매핑형

`collections.abc` 모듈은 `dict` 및 이와 유사한 자료형의 인터페이스를 정의하기 위해 `Mapping` 및 `MutableMapping` 추상 베이스 클래스를 제공한다.

In [1]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

표준 라이브러리에서 제공하는 매핑형은 모두 `dict`를 이용해서 구현하므로, 키가 __해시 가능__해야한다. 여기서 해시 가능하다는 말은 변하지 않는 해시값을 갖고 있고(`__hash__()` 메서드) 다른 객체와 비교할 수 있다(`__eq__()` 메서드)는 말이다. 원자적 불변형(str, byte, 수치형)은 모두 해시 가능하며, frozenset 또한 해시 가능하다. 튜플은 모든 항목들이 모두 해시 가능해야 해시 가능하다.

In [2]:
tt = (1,2,(30,40))
hash(tt)

8027212646858338501

In [3]:
tl = (1,2,[30,40])
hash(tl)

TypeError: unhashable type: 'list'

In [4]:
tf = (1,2,frozenset([30,40]))
hash(tf)

985328935373711578

딕셔너리는 다음과 같이 다양한 방법으로 구현할 수 있다.

In [5]:
a = dict(one=1, two=2)
a

{'one': 1, 'two': 2}

In [7]:
b = {'one':1, 'two':2}
b

{'one': 1, 'two': 2}

In [8]:
c = dict(zip(['one','two'],[1,2]))
c

{'one': 1, 'two': 2}

In [11]:
d = dict([('two',2),('one',1)])
d

{'two': 2, 'one': 1}

In [12]:
e = dict({'one':1, 'two':2})
e

{'one': 1, 'two': 2}

In [13]:
a == b == c == d == e

True

# 3.2 지능형 딕셔너리

In [15]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (55, 'Brazil'),
]

In [16]:
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'Brazil': 55}

In [21]:
{code: country.upper() for country, code in country_code.items()
    if code < 70}

{55: 'BRAZIL'}

# 3.3 공통적인 매핑 메서드

`d.update(m, [**kargs])` 메서드는 (키, 값) 쌍의 매핑이나 반복형 객체에서 가져온 항목들로 d를 갱신한다. 이 메서드가 첫 번째 인수 m을 다루는 방식은 덕 타이핑의 대표적인 사례이다.

1. 먼저 m이 `keys()` 메서드를 갖고 있으면 매핑이라고 간주한다.
1. 메서드가 없으면 `update()` 메서드는 m의 항목들이 (키, 값) 쌍으로 되어 있다고 간주하고 m을 반복한다.

## 3.3.1 존재하지 않는 키를 setdefault()로 처리하기

존재하지 않는 키 k로 `d[k]`를 접근하면 조기 실패 철학에 따라 `dict`는 오류를 발생시킨다. 이 경우 `d[k]` 대신 `d.get(k, default)`를 사용하면 대신 `default`를 반환하도록 할 수 있지만, 이는 효율성이 좋지 않다.

In [29]:
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
def index0(txt_file):
    import sys, re

    WORD_RE = re.compile(r'\w+')
    index = {}

    with open(txt_file, encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                occurence = index.get(word, [])
                occurence.append(location)
                index[word] = occurence
                
    for word in sorted(index, key=str.upper):
        print(word, index[word])

In [28]:
index0('zen.txt')

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

위의 코드를 `dict.setdefault()`를 사용하여 한 줄로 수정할 수 있다.

> `dict.setdefault()` : k in d가 참이면 `d[k]`를 반환하고, 아니면 `d[k]=default`로 설정하고 이 값을 반환한다.

In [32]:
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
def index1(txt_file):
    import sys, re

    WORD_RE = re.compile(r'\w+')
    index = {}

    with open(txt_file, encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                index.setdefault(word, []).append(location)
                
    for word in sorted(index, key=str.upper):
        print(word, index[word])

In [33]:
index1('zen.txt')

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

# 3.4 융퉁성 있게 키를 조회하는 매핑

검색할 때 키가 존재하지 않으면 어떤 특별한 값을 반환하는 매핑이 있으면 편리하다. 이는 

1. `dict` 대신 `defaultdict`를 쓰거나, 

1. `dict` 등의 매핑형을 상속해서 `__missing__()` 메서드를 추가하면 된다.